In [83]:
!ls dataPruebaDataEngineer
!pip install psycopg2

2012-1.csv  2012-2.csv	2012-3.csv  2012-4.csv	2012-5.csv  validation.csv
  Using cached psycopg2-2.9.3.tar.gz (380 kB)
  Created wheel for psycopg2: filename=psycopg2-2.9.3-cp39-cp39-linux_x86_64.whl size=168235 sha256=df7d1971120de150e543e8b862b210bf414ffc8b0112addf972773f5ff6aad2a
  Stored in directory: /home/cb4y/.cache/pip/wheels/b3/a1/6e/5a0e26314b15eb96a36263b80529ce0d64382540ac7b9544a9
Successfully built psycopg2


In [123]:
import os
import zipfile
import pandas as pd
import psycopg2
import statistics

# Conexión a la base de datos personal, 
# Deberia guardar esto como variables de entorno !OJO!
import sys

param_dic = {
    "host"      : "db-postgresql-nyc3-12814-do-user-10449879-0.b.db.ondigitalocean.com",
    "port"      : "25060",
    "database"  : "pragma",
    "user"      : "doadmin",
    "password"  : "lnwQqF6Aiv8qKn4I"
}

def select_database(conn, select_req):
    """
    Execute a single SELECT request
    :param conn: connection to database
    :param select_req: Select query to do
    in database
    :returns: data selected
    :raises DatabaseError: raises an exception
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_req)
        data = cursor.fetchall()
        conn.commit()
        return data
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error in select, DB error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()


def single_insert(conn, insert_req):
    """
    Execute a single INSERT request
    :param conn: connection to database
    :param select_req: Select query to do
    in database
    :returns: data selected
    :raises DatabaseError: raises an exception
    """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error in inserting in DB: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()


def connect(params_dic):
    """
    Connect to the PostgreSQL database server
    :param params_dic: params of connection
    :returns: connection to database
    :raises DatabaseError: raises an exception
    """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error in connection to DB.", error)
        sys.exit(1)
    return conn


def unzip_file(filepath, path_compressed_files):
    """
    Unzip a zip file
    :param filepath: string with the file path
    :param path_compressed_files: string with 
    the path with compressed files
    :returns: None
    :raises BadZipfile: raises an exception
    """
    try:
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(path_compressed_files)
    except BadZipfile as err:
        print("Error in unZipFile method: ", err)
        
def list_files_in_dir(directory):
    """
    List all files in a specific directory
    :param directory: string with the directory path
    :returns: a list of files inside of the directory
    :raises Exception: raises an exception
    """
    try:
        return os.listdir(directory)
    except Exception as err:
        print("Error listing files in dir: ", err)
        
def read_csv_to_pandas_df(filepath, columns=['timestamp',
                                             'price',
                                             'user_id']):
    """
    Read csv file and returns dataframe of pandas
    :param filepath: string with the directory path
    :param columns: array of the column names
    :returns: Pandas dataframe
    :raises Exception: raises an exception
    """
    try:
        df = pd.read_csv(filepath, usecols=columns)
        return df
    except Exception as err:
        print("Error leyendo el dataframe: ", err)


def is_csv_valid_file(file_to_check, extension, pattern):
    """Check if the dir have csv files to start pipeline
    :param file_to_check: String with the name of file to check
    :param extension: String with the extension to check
    :param pattern: String with pattern to check
    :returns: True is is valid, False in otherwise
    """
    if file_to_check.endswith(extension) and pattern in file_to_check:
        return True
    else:
        return False

In [124]:
if __name__ == "__main__":
    """Defino la configuración inicial del pipeline
    Nombre del archivo ZIP
    Nombre del directorio sin comprimir
    Extension de los archivos
    Patrón que debe cumplir los archivos
    """
    filepath_zip = "./dataPruebaDataEngineer (1).zip"
    folder_zip = './dataPruebaDataEngineer'
    extension = '.csv'
    pattern = '2012'
    # Descomprimo, listo y ordeno los archivos descomprimidos
    unzip_file(filepath_zip, '.')
    files = list_files_in_dir(folder_zip)
    files.sort()
    # Me conecto a la base de datos previamente configurada
    conn = connect(param_dic)
    # Empiezo a iterar sobre todos los archivos sin comprimir y cargo en memoria el dataframe
    # Unicamente si coincide con el patrón configurado
    for fil in files:
        if(is_csv_valid_file(fil, extension, pattern)):
            print(f"Scraping {fil}...")
            df = read_csv_to_pandas_df(folder_zip + '/' + fil)
    # Ordeno los tipos de datos y decido llenar los datos en price NaN con 0
    # Decidí dejar los valores nulos en 0, ya que supuse que la compra se realizo
    # pero no tuvieron valor, por lo que lo más adecuado fue dejarlo en 0
            df = df.fillna(0)
            df['timestamp']= pd.to_datetime(df['timestamp'])
            df['price'] = df['price'].astype(float)
            df['user_id'] = df['user_id'].astype('int64')
    #       Obtengo el min, max y media de la Series price de cada archivo
    #       No lo imprimo solo me sirve de validación tambian
    #       df_min_price = df['price'].min()
    #       df_max_price = df['price'].max()
    #       Creo el contador para validar los registros correctos en la DB
    #       Lo inicializo en 0, junto con una lista vacia que voy a usar para
    #       calcular la media, el minimo y el maximo del precio
            count_correct_insert = 0
            price_list = []
#           Empiezo a iterar sobre el dataframe cargado en memoria
            for i in df.index:
                try:
#                 Genero un query con cada uno de los valores del Dataframe y lo inserto
#                 en la base de datos y si es exitoso lo le sumo uno al contador e inserto 
#                 el precio actual en la lista del dataframe
                    query = f"""
                    INSERT into sales(user_id, filename, price, mark_time)
                    VALUES('{df.iloc[i]['user_id']}', '{fil}', '{df.iloc[i]['price']}', 
                        '{df.iloc[i]['timestamp']}');
                    """
                    single_insert(conn, query)
                    count_correct_insert += 1
                    price_list.append(df.iloc[i]['price'])
#                 Saco la media de la lista en la que tengo el precio de cada registro
#                 e imprimo el mensaje con el valor medio, el minimo y el máximo, 
#                 y finalemnte cierro la conexión cuando termine de recorrer todos los archivos
                    media = statistics.mean(price_list)
                    message = f"""
                    Llevamos: {count_correct_insert} registros insertados de: {len(df.index)},
                    Media de precio: {media},
                    Valor precio mínimo: {min(price_list)}
                    Valor precio máximo: {max(price_list)}
                    """
                    print(message)
                except Exception as err:
                    print("Error insertando: ", err)
    conn.close()

Connecting to the PostgreSQL database...
Scraping 2012-1.csv...

                    Llevamos: 1 registros insertados de: 22,
                    Media de precio: 50.0,
                    Valor precio mínimo: 50.0
                    Valor precio máximo: 50.0
                    

                    Llevamos: 2 registros insertados de: 22,
                    Media de precio: 68.5,
                    Valor precio mínimo: 50.0
                    Valor precio máximo: 87.0
                    

                    Llevamos: 3 registros insertados de: 22,
                    Media de precio: 67.0,
                    Valor precio mínimo: 50.0
                    Valor precio máximo: 87.0
                    

                    Llevamos: 4 registros insertados de: 22,
                    Media de precio: 55.25,
                    Valor precio mínimo: 20.0
                    Valor precio máximo: 87.0
                    

                    Llevamos: 5 registros insertados de: 22,
 


                    Llevamos: 15 registros insertados de: 29,
                    Media de precio: 61.666666666666664,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 100.0
                    

                    Llevamos: 16 registros insertados de: 29,
                    Media de precio: 60.125,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 100.0
                    

                    Llevamos: 17 registros insertados de: 29,
                    Media de precio: 58.23529411764706,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 100.0
                    

                    Llevamos: 18 registros insertados de: 29,
                    Media de precio: 55.55555555555556,
                    Valor precio mínimo: 10.0
                    Valor precio máximo: 100.0
                    

                    Llevamos: 19 registros insertados de: 29,
               


                    Llevamos: 23 registros insertados de: 31,
                    Media de precio: 58.869565217391305,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 99.0
                    

                    Llevamos: 24 registros insertados de: 31,
                    Media de precio: 58.666666666666664,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 99.0
                    

                    Llevamos: 25 registros insertados de: 31,
                    Media de precio: 59.6,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 99.0
                    

                    Llevamos: 26 registros insertados de: 31,
                    Media de precio: 60.15384615384615,
                    Valor precio mínimo: 12.0
                    Valor precio máximo: 99.0
                    

                    Llevamos: 27 registros insertados de: 31,
                    


                    Llevamos: 29 registros insertados de: 30,
                    Media de precio: 52.793103448275865,
                    Valor precio mínimo: 0.0
                    Valor precio máximo: 97.0
                    

                    Llevamos: 30 registros insertados de: 30,
                    Media de precio: 53.56666666666667,
                    Valor precio mínimo: 0.0
                    Valor precio máximo: 97.0
                    
Scraping 2012-5.csv...

                    Llevamos: 1 registros insertados de: 31,
                    Media de precio: 82.0,
                    Valor precio mínimo: 82.0
                    Valor precio máximo: 82.0
                    

                    Llevamos: 2 registros insertados de: 31,
                    Media de precio: 71.5,
                    Valor precio mínimo: 61.0
                    Valor precio máximo: 82.0
                    

                    Llevamos: 3 registros insertados de: 31,
                